In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import FormatStrFormatter
# import seaborn as sns
# sns.set_theme(context="poster", font_scale=1)

from pathlib import Path

sys.path.append("/home/acho/Sync/KiddLab/MSM/src")
from utils.stim_tools import *

|| MSM INITIALIZATION
Setting paths...
Importing sigtools...
Setting up sounddevice...
Importing level adjustments...
Setting eligible BUG words...
Preparing to create long-term spectrum matched noise...
Set values for tone pattern synthesis...
Initialization complete!


# Pilot v7

In [14]:
stim_df = pd.read_csv(PROJ_DIR/"assets"/"stimuli"/"stimulus_database.csv")
run0 = pd.read_csv(DATA_DIR/"RUN_000.csv")
run1 = pd.read_csv(DATA_DIR/"RUN_001.csv")
all_runs = pd.concat([run0, run1], ignore_index=True)
all_df = pd.merge(all_runs, stim_df, how="inner")

# Pilot v6

In [ ]:
def extract_condition_data(merged_data, cond):
    from functools import partial, reduce
    inner_merge = partial(pd.merge, how="inner")
    flatten = lambda t: [item for sublist in t for item in sublist] # flattens list of lists in double loop

    # Choose the subset of the stimulus database that satisfies the current conditions
    conditions = []
    conditions.append(merged_data[(merged_data["stim_type"] == cond.stim_type)])
    conditions.append(merged_data[(merged_data["is_target"] == True) &
                                  (merged_data["alt_rate"] == cond.target_alt_rate)])
    conditions.append(merged_data[(merged_data["is_target"] == False) &
                                  (merged_data["alt_rate"] == cond.masker_alt_rate)])
    if cond.target_init_angle: # if initial angle is specified in conditions, it will be not None
        conditions.append(merged_data[(merged_data["is_target"] == True) &
                                      (np.abs(merged_data["init_angle"]) == cond.target_init_angle)])
    if cond.masker_init_angle:
        conditions.append(merged_data[(merged_data["is_target"] == False) &
                                      (np.abs(merged_data["init_angle"]) == cond.masker_init_angle)])
    subsets = [merged_data.loc[merged_data["stim_num"].isin(cond["stim_num"])]
               for cond in conditions]
    conditioned_data = reduce(inner_merge, subsets)
    conditioned_data = conditioned_data[conditioned_data["is_target"]]
    return conditioned_data

In [ ]:
stim_data = pd.read_csv(PROJ_DIR/"assets"/"stimuli"/"stimulus_database.csv")
runs_to_plot = sorted(list(DATA_DIR.glob("*.csv")))

run_storage = pd.DataFrame()
for run_csv in runs_to_plot:
    run_storage = run_storage.append(pd.read_csv(run_csv))
merged_data = pd.merge(run_storage, stim_data, how="inner").reset_index(drop=True)

subject_list = np.sort(merged_data["subject_ID"].unique())
subject_list = [subj for subj in subject_list if "TRAIN" not in subj]
symbols = ["o", "s", "^", "v"]
subj_symb_map = dict(zip(subject_list, symbols[:len(subject_list)]))

control_em = [extract_condition_data(merged_data, cond) for cond in [PILOT_V6_CONTROLS[0]]]
expt_ev_em = [extract_condition_data(merged_data, cond) for cond in PILOT_V6_EV_EM]
expt_dv_em = [extract_condition_data(merged_data, cond) for cond in PILOT_V6_DV_EM]
control_im = [extract_condition_data(merged_data, cond) for cond in [PILOT_V6_CONTROLS[1]]]
expt_ev_im = [extract_condition_data(merged_data, cond) for cond in PILOT_V6_EV_IM]
expt_dv_im = [extract_condition_data(merged_data, cond) for cond in PILOT_V6_DV_IM]

In [ ]:
# Plot parameters
chance = 1/8

cyan   = (  0/256, 173/256, 239/256, 0.75)
violet = (128/256,   0/256, 128/256, 0.75)

x_min, x_max, x_spacing = -0.25, 4.25, 1
y_min, y_max, y_spacing = 0, 1, 0.1
x_ticks = np.arange(0, 5, x_spacing)
y_ticks = np.arange(y_min + y_spacing, y_max, y_spacing)

dv0 = r"""Static
"""
dv1 = r"""0.1
"""
dv2 = r"""0.5
"""
dv3 = r"""1.0
"""
dv4 = r"""2.0
"""
ev0 = r"""Static
"""
ev1 = r"""0.1
"""
ev2 = r"""0.5
"""
ev3 = r"""1.0
"""
ev4 = r"""2.0
"""
dv_x_ticklabels = [dv0, dv1, dv2, dv3, dv4]
ev_x_ticklabels = [ev0, ev1, ev2, ev3, ev4]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 12), constrained_layout=True)

ev_em = np.empty((len(subject_list), 5))
dv_em = np.empty((len(subject_list), 5))
ev_im = np.empty((len(subject_list), 5))
dv_im = np.empty((len(subject_list), 5))
for subj in subject_list:
    curr_subj_control_em = [df[df["subject_ID"] == subj] for df in control_em]
    curr_subj_expt_ev_em = [df[df["subject_ID"] == subj] for df in expt_ev_em]
    curr_subj_expt_dv_em = [df[df["subject_ID"] == subj] for df in expt_dv_em]
    curr_subj_control_im = [df[df["subject_ID"] == subj] for df in control_im]
    curr_subj_expt_ev_im = [df[df["subject_ID"] == subj] for df in expt_ev_im]
    curr_subj_expt_dv_im = [df[df["subject_ID"] == subj] for df in expt_dv_im]
    curr_control_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_control_em])
    curr_expt_ev_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_ev_em])
    curr_expt_dv_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_dv_em])
    curr_control_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_control_im])
    curr_expt_ev_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_ev_im])
    curr_expt_dv_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_dv_im])

    axes[0, 0].plot(x_ticks[0 ], curr_control_em_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 0].plot(x_ticks[1:], curr_expt_ev_em_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 0].plot(x_ticks[0 ], curr_control_em_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 0].plot(x_ticks[1:], curr_expt_dv_em_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 1].plot(x_ticks[0 ], curr_control_im_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 1].plot(x_ticks[1:], curr_expt_ev_im_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 1].plot(x_ticks[0 ], curr_control_im_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 1].plot(x_ticks[1:], curr_expt_dv_im_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)

fig.suptitle(r"\textbf{Word identification performance}", fontsize=24)
axes[0, 0].set_title(r"(Noise Masker)",  fontsize=18)
axes[0, 1].set_title(r"(Speech Masker)", fontsize=18)
for i, ax_rows in enumerate(axes):
    for j, ax in enumerate(ax_rows):
        ax.set_xticks(x_ticks)
        if i == 0:
#             ax.set_xticklabels(dv_x_ticklabels, fontsize=12)
            ax.set_xticklabels([], fontsize=12)
        else:
            ax.set_xticklabels(ev_x_ticklabels, fontsize=12)
        ax.set_yticks(y_ticks)
        if j == 0:
            ax.set_yticklabels(["{:.1f}".format(i) for i in y_ticks], fontsize=16)
        else:
            ax.set_yticklabels([])
        ax.tick_params(axis="both", length=0)
        ax.grid(linestyle="-", alpha=0.8)
        ax.hlines(chance, x_min, x_max, colors="k", linestyles=":")
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
fig.text( 0.5 , -0.01, r"\textbf{Target rate [Hz]}", ha="center", rotation="horizontal", fontsize=18)
fig.text(-0.03,  0.5 , r"\textbf{Proportion correct}",  va="center", rotation="vertical",   fontsize=18)

plt.show()
# fig.savefig("data_pilot_v6.pdf", bbox_inches="tight")

# Pilot v5

Load data and calculate desired dataframes:

In [ ]:
stim_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v5"/"stimuli"/"stimulus_database.csv")
runs_to_plot = sorted(list((PROJ_DIR/"archive"/"pilot_v5"/"data").glob("*.csv")))

run_storage = pd.DataFrame()
for run_csv in runs_to_plot:
    run_storage = run_storage.append(pd.read_csv(run_csv))
merged_data = pd.merge(run_storage, stim_data, how="inner").reset_index(drop=True)

subject_list = np.sort(merged_data["subject_ID"].unique())
subject_list = [subj for subj in subject_list if "TRAIN" not in subj]
symbols = ["o", "s", "^", "v"]
subj_symb_map = dict(zip(subject_list, symbols[:len(subject_list)]))




CONTROL_EM = [Condition("SEM", 0. , 0. , 45. , 45. ),
              Condition("SEM", 0. , 0. , 63.5, 63.5)]
EXPT_EV_EM = [Condition("SEM", 0.5, 0.5, None, None),
              Condition("SEM", 1. , 1. , None, None),
              Condition("SEM", 2. , 2. , None, None),
              Condition("SEM", 5. , 5. , None, None)]
EXPT_DV_EM = [Condition("SEM", 0.5, 5. , None, None),
              Condition("SEM", 1. , 2. , None, None),
              Condition("SEM", 2. , 1. , None, None),
              Condition("SEM", 5. , 0.5, None, None)]
CONTROL_IM = [Condition("SIM", 0. , 0. , 45. , 45. ),
              Condition("SIM", 0. , 0. , 63.5, 63.5)]
EXPT_EV_IM = [Condition("SIM", 0.5, 0.5, None, None),
              Condition("SIM", 1. , 1. , None, None),
              Condition("SIM", 2. , 2. , None, None),
              Condition("SIM", 5. , 5. , None, None)]
EXPT_DV_IM = [Condition("SIM", 0.5, 5. , None, None),
              Condition("SIM", 1. , 2. , None, None),
              Condition("SIM", 2. , 1. , None, None),
              Condition("SIM", 5. , 0.5, None, None)]
TRAIN_COND = [Condition("SEM", 0. , 0. , 63.5, 63.5),
              Condition("SEM", 1. , 1. , None, None),
              Condition("SEM", 1,   2. , None, None),
              Condition("SEM", 0. , 0. , 45. , 45. ),
              Condition("SIM", 0.5, 0.5, None, None),
              Condition("SIM", 0.5, 5. , None, None)]
ALL_CONDITIONS = CONTROL_EM + EXPT_EV_EM + EXPT_DV_EM \
               + CONTROL_IM + EXPT_EV_IM + EXPT_DV_IM

control_em = [extract_condition_data(merged_data, cond) for cond in CONTROL_EM]
expt_ev_em = [extract_condition_data(merged_data, cond) for cond in EXPT_EV_EM]
expt_dv_em = [extract_condition_data(merged_data, cond) for cond in EXPT_DV_EM]
control_im = [extract_condition_data(merged_data, cond) for cond in CONTROL_IM]
expt_ev_im = [extract_condition_data(merged_data, cond) for cond in EXPT_EV_IM]
expt_dv_im = [extract_condition_data(merged_data, cond) for cond in EXPT_DV_IM]

In [ ]:
# Plot parameters
chance = 1/8

cyan   = (  0/256, 173/256, 239/256, 0.75)
violet = (128/256,   0/256, 128/256, 0.75)

x_min, x_max, x_spacing = -0.25, 4.25, 1
y_min, y_max, y_spacing = 0, 1, 0.1
x_ticks = np.arange(0, 5, x_spacing)
y_ticks = np.arange(y_min + y_spacing, y_max, y_spacing)

dv0 = r"""T/ST
M/ST
($\Delta\theta=127^{\circ}$)
"""
dv1 = r"""T/VS
M/VS
"""
dv2 = r"""T/SL
M/SL
"""
dv3 = r"""T/FS
M/FS
"""
dv4 = r"""T/VF
M/VF
"""
ev0 = r"""T/ST
M/ST
($\Delta\theta=90^{\circ}$)
"""
ev1 = r"""T/VS
M/VF
"""
ev2 = r"""T/SL
M/FS
"""
ev3 = r"""T/FS
M/SL
"""
ev4 = r"""T/VF
M/VS
"""
dv_x_ticklabels = [dv0, dv1, dv2, dv3, dv4]
ev_x_ticklabels = [ev0, ev1, ev2, ev3, ev4]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 12), constrained_layout=True)

ev_em = np.empty((len(subject_list), 5))
dv_em = np.empty((len(subject_list), 5))
ev_im = np.empty((len(subject_list), 5))
dv_im = np.empty((len(subject_list), 5))
for subj in subject_list:
    curr_subj_control_em = [df[df["subject_ID"] == subj] for df in control_em]
    curr_subj_expt_ev_em = [df[df["subject_ID"] == subj] for df in expt_ev_em]
    curr_subj_expt_dv_em = [df[df["subject_ID"] == subj] for df in expt_dv_em]
    curr_subj_control_im = [df[df["subject_ID"] == subj] for df in control_im]
    curr_subj_expt_ev_im = [df[df["subject_ID"] == subj] for df in expt_ev_im]
    curr_subj_expt_dv_im = [df[df["subject_ID"] == subj] for df in expt_dv_im]
    curr_control_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_control_em])
    curr_expt_ev_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_ev_em])
    curr_expt_dv_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_dv_em])
    curr_control_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_control_im])
    curr_expt_ev_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_ev_im])
    curr_expt_dv_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_dv_im])

    axes[0, 0].plot(x_ticks[0 ], curr_control_em_corr[1], color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 0].plot(x_ticks[1:], curr_expt_ev_em_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 0].plot(x_ticks[0 ], curr_control_em_corr[0], color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 0].plot(x_ticks[1:], curr_expt_dv_em_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 1].plot(x_ticks[0 ], curr_control_im_corr[1], color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 1].plot(x_ticks[1:], curr_expt_ev_im_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 1].plot(x_ticks[0 ], curr_control_im_corr[0], color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 1].plot(x_ticks[1:], curr_expt_dv_im_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)

fig.suptitle(r"\textbf{Word identification performance}", fontsize=24)
axes[0, 0].set_title(r"(Noise Masker)",  fontsize=18)
axes[0, 1].set_title(r"(Speech Masker)", fontsize=18)
for i, ax_rows in enumerate(axes):
    for j, ax in enumerate(ax_rows):
        ax.set_xticks(x_ticks)
        if i == 0:
            ax.set_xticklabels(dv_x_ticklabels, fontsize=12)
        else:
            ax.set_xticklabels(ev_x_ticklabels, fontsize=12)
        ax.set_yticks(y_ticks)
        if j == 0:
            ax.set_yticklabels(["{:.1f}".format(i) for i in y_ticks], fontsize=16)
        else:
            ax.set_yticklabels([])
        ax.tick_params(axis="both", length=0)
        ax.grid(linestyle="-", alpha=0.8)
        ax.hlines(chance, x_min, x_max, colors="k", linestyles=":")
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
fig.text( 0.5 , -0.01, r"\textbf{Movement conditions}", ha="center", rotation="horizontal", fontsize=18)
fig.text(-0.03,  0.5 , r"\textbf{Proportion correct}",  va="center", rotation="vertical",   fontsize=18)

plt.show()
# fig.savefig("data_pilot_v5.pdf", bbox_inches="tight")

# Pilot v4

In [ ]:
stim_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v4"/"stimuli"/"stimulus_database.csv")
runs_to_plot = range(7)

run_storage = pd.DataFrame()
for run_num in runs_to_plot:
    run_file_name = "RUN_" + str(run_num).zfill(3) + ".csv"
    run_storage = run_storage.append(pd.read_csv(DATA_DIR/run_file_name))
merged_data = pd.merge(run_storage, stim_data, how="inner").reset_index(drop=True)

In [ ]:
def extract_condition_data(merged_data, cond):
    from functools import partial, reduce
    inner_merge = partial(pd.merge, how="inner")
    flatten = lambda t: [item for sublist in t for item in sublist] # flattens list of lists in double loop

    # Choose the subset of the stimulus database that satisfies the current conditions
    conditions = []
    conditions.append(merged_data[(merged_data["stim_type"] == cond.stim_type)])
    conditions.append(merged_data[(merged_data["is_target"] == True) &
                                  (merged_data["alt_rate"] == cond.target_alt_rate)])
    conditions.append(merged_data[(merged_data["is_target"] == False) &
                                  (merged_data["alt_rate"] == cond.masker_alt_rate)])
    if cond.target_init_angle: # if initial angle is specified in conditions, it will be not None
        conditions.append(merged_data[(merged_data["is_target"] == True) &
                                      (np.abs(merged_data["init_angle"]) == cond.target_init_angle)])
    if cond.masker_init_angle:
        conditions.append(merged_data[(merged_data["is_target"] == False) &
                                      (np.abs(merged_data["init_angle"]) == cond.masker_init_angle)])
    subsets = [merged_data.loc[merged_data["stim_num"].isin(cond["stim_num"])]
               for cond in conditions]
    conditioned_data = reduce(inner_merge, subsets)
    conditioned_data = conditioned_data[conditioned_data["is_target"]]
    return conditioned_data

In [ ]:
expt111_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_1_1]
expt112_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_1_2]
expt121_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_2_1]
expt122_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_2_2]
expt131_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_3_1]
expt132_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_3_2]
expt212_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_2_1_2]

expt111_corr = np.array([(data["correct"]/4).mean() for data in expt111_data_by_cond])
expt112_corr = np.array([(data["correct"]/4).mean() for data in expt112_data_by_cond])
expt121_corr = np.array([(data["correct"]/4).mean() for data in expt121_data_by_cond])
expt122_corr = np.array([(data["correct"]/4).mean() for data in expt122_data_by_cond])
expt131_corr = np.array([(data["correct"]/4).mean() for data in expt131_data_by_cond])
expt132_corr = np.array([(data["correct"]/4).mean() for data in expt132_data_by_cond])
expt212_corr = np.array([(data["correct"]/4).mean() for data in expt212_data_by_cond])

In [ ]:
legend0 = []
legend0.append(Patch(facecolor="r", edgecolor="r", label="Antiphasic"))
legend0.append(Patch(facecolor="b", edgecolor="b", label="Static target"))
legend0.append(Patch(facecolor="g", edgecolor="g", label="Static masker"))
legend0.append(Line2D([], [], linestyle="", marker="d", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="w", markersize=12,
                      label="Noise masker"))
legend0.append(Line2D([], [], linestyle="", marker="s", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="w", markersize=12,
                      label="Speech masker"))
legend0.append(Line2D([], [], linestyle="", marker="^", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="k", markersize=12,
                      label=r"$\pm 90^{\circ}$"))
legend0.append(Line2D([], [], linestyle="", marker="v", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="k", markersize=12,
                      label=r"$0^{\circ}$"))
legend0.append(Line2D([], [], linestyle="", marker="o", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="k", markersize=12,
                      label="0.5 vs 5 Hz"))

# legend_elements.append(Patch(facecolor="b", edgecolor="b", label=r"$\pm 90^{\circ}$"),)
# legend_elements.append(Patch(facecolor="r", edgecolor="r", label=r"$0^{\circ}$"))
# legend_elements.append(Line2D([], [], color="k", linestyle=":", label="chance"))

In [ ]:
delta_w = 0.1
pilot_v3_pm = [0.9875, 0.9625]
pilot_v3_coloc = [0.85, 0.775]
x_locs = np.arange(1, 5)
y_locs = np.arange(0.5, 1.01, 0.1)
x_ticklabels = [0.5, 1, 2, 5]
y_ticklabels = ["{:.1f}".format(y_val) for y_val in y_locs]
y_ticks_ax1 = np.linspace(-0.3, 0.3, 7, endpoint=True)
y_ticklabels_ax1 = ["{:+.1f}".format(y_val) for y_val in y_ticks_ax1]

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
axes[0].plot([3*delta_w, 3*delta_w], pilot_v3_pm, "k^", markersize=12, alpha=0.75)
axes[0].plot([4*delta_w, 4*delta_w], pilot_v3_coloc, "kv", markersize=12, alpha=0.75)
axes[0].plot([5*delta_w, 5*delta_w], expt212_corr , "ko", markersize=12, alpha=0.75)
axes[0].plot(x_locs - 1*delta_w, expt111_corr, "rd--", markersize=12, alpha=0.75)
axes[0].plot(x_locs + 0*delta_w, expt121_corr, "bd--", markersize=12, alpha=0.75)
axes[0].plot(x_locs + 1*delta_w, expt131_corr, "gd--", markersize=12, alpha=0.75)
axes[0].plot(x_locs - 1*delta_w, expt112_corr, "rs:",  markersize=12, alpha=0.75)
axes[0].plot(x_locs + 0*delta_w, expt122_corr, "bs:",  markersize=12, alpha=0.75)
axes[0].plot(x_locs + 1*delta_w, expt132_corr, "gs:",  markersize=12, alpha=0.75)

axes[0].set_title("Words correct - one target, one masker", fontsize=26)
axes[0].set_xlabel("Oscillation rate [Hz]", fontsize=24)
axes[0].set_ylabel("Proportion correct", fontsize=24)
axes[0].set_xlim((0, x_locs[-1] + 5*delta_w))
axes[0].set_ylim((0.5, 1.01))
axes[0].set_xticks(x_locs)
axes[0].set_yticks(y_locs)
axes[0].set_xticklabels(x_ticklabels, fontsize=20)
axes[0].set_yticklabels(y_ticklabels, fontsize=20)
axes[0].grid(color="k", linestyle="-", alpha=0.25)
axes[0].legend(handles=legend0, loc=0, fontsize=18)

axes[1].bar(x_locs + 0*delta_w, expt112_corr - expt111_corr,
            width=delta_w, color="r", align="edge", alpha=0.75)
axes[1].bar(x_locs + 1*delta_w, expt122_corr - expt121_corr,
            width=delta_w, color="b", align="edge", alpha=0.75)
axes[1].bar(x_locs + 2*delta_w, expt132_corr - expt131_corr,
            width=delta_w, color="g", align="edge", alpha=0.75)
axes[1].hlines(0, 1*delta_w, x_locs[-1] + 4*delta_w, color="k", alpha=0.8)

axes[1].set_title("How much easier is SOS vs noise? (i.e. EM vs IM)", fontsize=26)
axes[1].set_xlabel("Oscillation rate [Hz]", fontsize=24)
axes[1].set_ylabel(r"$\Delta$ prop. correct", fontsize=24)
axes[1].set_xlim((x_locs[0] - 1*delta_w, x_locs[-1] + 4*delta_w))
axes[1].set_ylim((-0.31, 0.31))
axes[1].set_xticks(x_locs + 1.5*delta_w)
axes[1].set_yticks(y_ticks_ax1)
axes[1].set_xticklabels(x_ticklabels, fontsize=20)
axes[1].set_yticklabels(y_ticklabels_ax1, fontsize=20)
axes[1].grid(color="k", linestyle="-", alpha=0.25)

fig.tight_layout()
fig.savefig("pilot_v4.pdf")

# Pilot v3 - two or three talkers; also examining randomization effect
* Red - two talkers, conditions (alternation rates) are blocked
* Blue - two talkers, conditions are randomized within block
* Yellow - three talkers, conditions are blocked
* Green - three talkers, conditions are randomized within block

In [ ]:
stim_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v3"/"stimuli"/"stimulus_database.csv")
runs_to_plot = range(4)

run_storage = pd.DataFrame()
for run_num in runs_to_plot:
    run_file_name = "RUN_" + str(run_num).zfill(3) + ".csv"
    run_storage = run_storage.append(pd.read_csv(PROJ_DIR/"archive"/"pilot_v3"/"data"/run_file_name))
merged_data = pd.merge(run_storage, stim_data, how="inner").reset_index(drop=True)

In [ ]:
chance_level = 1/8
colors = ["r", "b", "y", "g", "m", "c", "y", "k"]
marker_symbols = ["o", "v", "^", "d", "*", "x", "s", "p"]
conditions = ["colocated", "plus_minus_90", 0.5, 2, 4, 8]
x_tick_locs = [-0.5, 9, 0.5, 2, 4, 8]
x_tick_labels = [(r"$\pm90^{\circ}$"), "co-loc", "0.5", "2", "4", "8"]

# legend_elements = []
# for i in range(n_runs_to_load):
#     legend_elements = [Line2D([], [], linestyle="", marker=marker_symbols[i], markeredgewidth=3, markeredgecolor="k", markerfacecolor="w", markersize=18, label=subjects[i])]
# legend_elements.append(Patch(facecolor="b", edgecolor="b", label=r"$\pm 90^{\circ}$"),)
# legend_elements.append(Patch(facecolor="r", edgecolor="r", label=r"$0^{\circ}$"))
# legend_elements.append(Line2D([], [], color="k", linestyle=":", label="chance"))

fig, ax = plt.subplots(1, 1, figsize=(12, 9))

for idx, run_num in enumerate(runs_to_plot):
    curr_run = merged_data[(merged_data["is_target"] == True) &
                           (merged_data["run_num"] == run_num)]
    rate_correct = []
    for cond in conditions:
        if cond == "colocated":
            curr_cond = curr_run[(curr_run["alt_rate"] == 0) &
                                 (curr_run["init_angle"] == 0)]
            n_correct = sum(curr_cond["correct"])
            n_total = 4*len(curr_cond)
            ax.plot(x_tick_locs[1], n_correct/n_total, colors[idx] + marker_symbols[idx])
        elif cond == "plus_minus_90":
            curr_cond = curr_run[(curr_run["alt_rate"] == 0) &
                                 (curr_run["init_angle"] != 0)]
            n_correct = sum(curr_cond["correct"])
            n_total = 4*len(curr_cond)
            ax.plot(x_tick_locs[0], n_correct/n_total, colors[idx] + marker_symbols[idx])
        else:
            curr_cond = curr_run[curr_run["alt_rate"] == cond]
            n_correct = sum(curr_cond["correct"])
            n_total = 4*len(curr_cond)
            rate_correct.append((cond, n_correct/n_total))
    rate_correct = np.array(rate_correct)
    ax.plot(rate_correct[:, 0], rate_correct[:, 1], colors[idx] + marker_symbols[idx] + "-")
ax.hlines(chance_level, x_tick_locs[0] - 0.5, x_tick_locs[1] + 0.5, color="k", linestyle=":", alpha=0.75)

ax.set_title("Word by word identification performance", fontsize=40)
ax.set_xlabel("Oscillation rate [Hz]", fontsize=28)
ax.set_ylabel("Proportion correct", fontsize=28)
ax.set_xlim((x_tick_locs[0] - 0.5, x_tick_locs[1] + 0.5))
ax.set_ylim((0, 1))
ax.set_xticks(x_tick_locs)
ax.set_yticks(np.linspace(0, 1, 11, endpoint=True))
ax.set_xticklabels(x_tick_labels, fontsize=20)
ax.set_yticklabels(np.linspace(0, 1, 11, endpoint=True), fontsize=20)
ax.yaxis.set_major_formatter(FormatStrFormatter("%.1f"))

ax.grid(linestyle="-")
# ax.legend(handles=legend_elements)

# plt.savefig("ayc_pilot_v3.pdf")
plt.show()

### Plot proportion correct by talker

In [ ]:
grouped_by_talker_2 = merged_data[(merged_data["is_target"] == True) &
                                  (merged_data["n_srcs"] == 2)].groupby("src").sum("correct")
correct_by_talker_2 = grouped_by_talker_2["correct"]
possible_correct_2 = 4*grouped_by_talker_2["is_target"].values
prop_correct_by_talker_2 = correct_by_talker_2/possible_correct_2
prop_correct_by_talker_2 = prop_correct_by_talker_2.sort_values(ascending=False)

grouped_by_talker_3 = merged_data[(merged_data["is_target"] == True) &
                                  (merged_data["n_srcs"] == 3)].groupby("src").sum("correct")
correct_by_talker_3 = grouped_by_talker_3["correct"]
possible_correct_3 = 4*grouped_by_talker_3["is_target"].values
prop_correct_by_talker_3 = correct_by_talker_3/possible_correct_3
prop_correct_by_talker_3 = prop_correct_by_talker_3.sort_values(ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
prop_correct_by_talker_2.plot(kind="bar", ylim=(0, 1), ax=axes[0])
prop_correct_by_talker_3.plot(kind="bar", ylim=(0, 1), ax=axes[1])

axes[0].set_title("Two talkers")
axes[1].set_title("Three talkers")
axes[1].set_yticklabels([])

plt.tight_layout()
# plt.show()

# Pilot v2 - three talkers, one co-located masker, one antiphasic masker

In [ ]:
run_figsize = (12, 8)
n_runs_to_load = 1
n_trials_per_block = 8
n_words_per_trial = 4
chance_level = 1/8

marker_symbols = ["o", "v", "^", "d", "*"]
subjects = ["ayc"]
legend_elements = []
for i in range(n_runs_to_load):
    legend_elements = [Line2D([], [], linestyle="", marker=marker_symbols[i], markeredgewidth=3, markeredgecolor="k", markerfacecolor="w", markersize=18, label=subjects[i])]
legend_elements.append(Patch(facecolor="b", edgecolor="b", label=r"$\pm 90^{\circ}$"),)
legend_elements.append(Patch(facecolor="r", edgecolor="r", label=r"$0^{\circ}$"))
legend_elements.append(Line2D([], [], color="k", linestyle=":", label="chance"))

all_run_nums = range(n_runs_to_load)
fig, ax = plt.subplots(1, 1, figsize=run_figsize)
for run_num in all_run_nums:
    run_file_name = "RUN_" + str(run_num).zfill(3) + ".csv"
    stim_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v2"/"stimuli"/"stimulus_database.csv")
    run_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v2"/"data"/run_file_name)
    
    run_stim = stim_data.loc[run_data.stimulus_ID].reset_index()
    run_stim = run_stim.drop(labels=["index", "stim_type"], axis=1)
    run_data = run_data.join(run_stim)
    run_data = run_data.drop(labels=["run_num", "subject_ID", "task_type"], axis=1)

    n_blocks = run_data.block_num.max()
#     n_max_correct = n_trials_per_block*n_words_per_trial
    n_max_correct = n_blocks*n_trials_per_block*n_words_per_trial

    # Group data
    run_rate_grouped = run_data.groupby(by="target_alt_rate")
    rates = list(run_rate_grouped.indices.keys())[1:]
    correct   = run_rate_grouped.sum()["correct"].values[1:]
    # either ear
    antipodal = run_data[(run_data["target_alt_rate"] == 0) & \
                         (run_data["target_init_position"] != 0)]["correct"].sum()
    # co-located
    colocated = run_data[(run_data["target_alt_rate"] == 0) & \
                         (run_data["target_init_position"] == 0)]["correct"].sum()

    # Plot
    ax.plot(rates[ 0] - 1, antipodal/n_max_correct, "b" + marker_symbols[run_num], markersize=18)
    ax.plot(        rates,   correct/n_max_correct, "k" + marker_symbols[run_num] + "-", markersize=18)
    ax.plot(rates[-1] + 1, colocated/n_max_correct, "r" + marker_symbols[run_num], markersize=18)
ax.hlines(chance_level, rates[0] - 2*1.75, rates[-1] + 2*1.75, color="k", linestyle=":")

ax.set_title("Word by word identification performance", fontsize=40)
ax.set_xlabel("Oscillation rate [Hz]\n(linear scale)", fontsize=28)
ax.set_ylabel("Percent correct", fontsize=28)
ax.set_xlim((rates[0] - 1.75, rates[-1] + 1.75))
ax.set_ylim((0, 1))
ax.set_xticks(rates)
ax.set_xticklabels(rates, rotation=90, fontsize=20)
ax.set_yticks(np.linspace(0, 1, 11, endpoint=True))
ax.set_yticklabels(np.linspace(0, 1, 11, endpoint=True), fontsize=20)
ax.xaxis.set_major_formatter(FormatStrFormatter("%.1f"))
ax.yaxis.set_major_formatter(FormatStrFormatter("%.1f"))
ax.grid(linestyle="-")
ax.legend(handles=legend_elements)
# ax.grid(linestyle="--", alpha=0.5)

# plt.savefig("b.pdf")
plt.show()